<a href="https://colab.research.google.com/github/premanshsharma/NLP-Project-Name/blob/main/AbusiveCommentDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd
link = '/content/gdrive/MyDrive/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Test_NoLabel.csv/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv'
df = pd.read_csv(link, on_bad_lines='skip')
list(df.columns.values)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3,4,5,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['CommentId',
 'commentText',
 'language',
 'user_index',
 'post_index',
 'report_count_comment',
 'report_count_post',
 'like_count_comment',
 'like_count_post',
 'label']

In [4]:
k =['CommentId', 'language', 'user_index', 'post_index','report_count_comment','report_count_post','like_count_comment','like_count_post']
for col in df.columns:
    if col in k:
        del df[col]
df

,commentText,label
0,randi ka deewana,1.0
1,👌👌🙏🥰🥰🥰☝️☝️,0.0
2,apni pant utar kar apni chut bhi to bata do ma...,1.0
3,krishnatripathi4578 jesi ghar ki sabhyata vais...,0.0
4,is randi ko dusra kaam nahi hai kuttiya kahin ki,1.0
...,...,...
1483501,pavam🤣 😁😻😻😻😻,0.0
1483502,enik ind post ittittila 😔,0.0
1483503,nirmelp34 parayilla,0.0
1483504,ettan🙈😘,0.0


In [15]:
data = []
result = []
labels = []
for heading in df:
    labels.append(heading)
n = len(df[labels[0]])
for i in range(n):
    if df[labels[1]][i] == 1.0 or df[labels[1]][i] == 0.0:
        data.append(df[labels[0]][i])
        result.append(int(df[labels[1]][i]))

In [16]:
import numpy as np